In [1]:
!pip install segmentation-models
!pip install tensorflow==2.8
!pip install keras==2.8


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [1]:
import sys

sys.path.append('c:/users/haris/appdata/roaming/python/python39/site-packages')

In [2]:
%matplotlib inline

In [3]:
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [5]:
import tensorflow as tf
import segmentation_models as sm
BACKBONE = 'efficientnetb0'
preprocess_input = sm.get_preprocessing(BACKBONE)

Segmentation Models: using `keras` framework.


In [6]:
sm.set_framework('tf.keras')

sm.framework()

'tf.keras'

In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 224
W = 224

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(dataset_path, split=0.1):
    images = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset17/ISIC-2017_Training_Data", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset17/ISIC-2017_Training_Part1_GroundTruth", "*.png")))

    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 32
    lr = 1e-4 ## (0.0001)
    num_epoch = 200
    model_path = "files/model.h5"
    csv_path = "files/data.csv"

    """ Dataset : 80/10/10 """
    dataset_path = "C:/Users/Haris/ICAI_Paper/Dataset17"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)


    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train: 1720 - 1720
Valid: 215 - 215
Test: 215 - 215
Epoch 1/200
54/54 [==============================] - ETA: 0s - loss: 0.5208 - dice_coef: 0.4151 - iou: 0.2649 - recall: 0.8201 - precision: 0.4390
Epoch 1: val_loss improved from inf to 0.99152, saving model to files\model.h5
54/54 [==============================] - 234s 4s/step - loss: 0.5208 - dice_coef: 0.4151 - iou: 0.2649 - recall: 0.8201 - precision: 0.4390 - val_loss: 0.9915 - val_dice_coef: 0.4174 - val_iou: 0.2660 - val_recall: 0.9894 - val_precision: 0.2309 - lr: 1.0000e-04
Epoch 2/200
54/54 [==============================] - ETA: 0s - loss: 0.3038 - dice_coef: 0.5682 - iou: 0.3986 - recall: 0.9294 - precision: 0.8127
Epoch 2: val_loss improved from 0.99152 to 0.42487, saving model to files\model.h5
54/54 [==============================] - 199s 4s/step - loss: 0.3038 - dice_coef: 0.5682 - iou: 0.3986 - recall: 0.9294 - precision: 0.8127 - val_loss: 0.4249 - val_dice_coef: 0.4674 - val_iou: 0.3068 - val_recall: 0.6521 - val_p

Epoch 18/200
54/54 [==============================] - ETA: 0s - loss: 0.0398 - dice_coef: 0.9223 - iou: 0.8560 - recall: 0.9689 - precision: 0.9729
Epoch 18: val_loss improved from 0.11222 to 0.10956, saving model to files\model.h5
54/54 [==============================] - 195s 4s/step - loss: 0.0398 - dice_coef: 0.9223 - iou: 0.8560 - recall: 0.9689 - precision: 0.9729 - val_loss: 0.1096 - val_dice_coef: 0.8727 - val_iou: 0.7745 - val_recall: 0.8914 - val_precision: 0.9346 - lr: 1.0000e-04
Epoch 19/200
54/54 [==============================] - ETA: 0s - loss: 0.0399 - dice_coef: 0.9242 - iou: 0.8594 - recall: 0.9682 - precision: 0.9703
Epoch 19: val_loss did not improve from 0.10956
54/54 [==============================] - 195s 4s/step - loss: 0.0399 - dice_coef: 0.9242 - iou: 0.8594 - recall: 0.9682 - precision: 0.9703 - val_loss: 0.1112 - val_dice_coef: 0.8754 - val_iou: 0.7788 - val_recall: 0.9048 - val_precision: 0.9199 - lr: 1.0000e-04
Epoch 20/200
54/54 [==========================

54/54 [==============================] - ETA: 0s - loss: 0.0269 - dice_coef: 0.9456 - iou: 0.8970 - recall: 0.9783 - precision: 0.9810
Epoch 35: val_loss did not improve from 0.10655
54/54 [==============================] - 196s 4s/step - loss: 0.0269 - dice_coef: 0.9456 - iou: 0.8970 - recall: 0.9783 - precision: 0.9810 - val_loss: 0.1098 - val_dice_coef: 0.8855 - val_iou: 0.7948 - val_recall: 0.8870 - val_precision: 0.9414 - lr: 1.0000e-07
Epoch 36/200
54/54 [==============================] - ETA: 0s - loss: 0.0269 - dice_coef: 0.9456 - iou: 0.8970 - recall: 0.9778 - precision: 0.9815
Epoch 36: val_loss did not improve from 0.10655
54/54 [==============================] - 195s 4s/step - loss: 0.0269 - dice_coef: 0.9456 - iou: 0.8970 - recall: 0.9778 - precision: 0.9815 - val_loss: 0.1097 - val_dice_coef: 0.8856 - val_iou: 0.7950 - val_recall: 0.8872 - val_precision: 0.9413 - lr: 1.0000e-07
Epoch 37/200
54/54 [==============================] - ETA: 0s - loss: 0.0272 - dice_coef: 0.945

In [8]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 stem_conv (Conv2D)             (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 stem_bn (BatchNormalization)   (None, None, None,   128         ['stem_conv[0][0]']              
                                32)                                                           

                                                                                                  
 block2b_expand_activation (Act  (None, None, None,   0          ['block2b_expand_bn[0][0]']      
 ivation)                       144)                                                              
                                                                                                  
 block2b_dwconv (DepthwiseConv2  (None, None, None,   1296       ['block2b_expand_activation[0][0]
 D)                             144)                             ']                               
                                                                                                  
 block2b_bn (BatchNormalization  (None, None, None,   576        ['block2b_dwconv[0][0]']         
 )                              144)                                                              
                                                                                                  
 block2b_a

 D)                             240)                             ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, None, None,   960        ['block3b_dwconv[0][0]']         
 )                              240)                                                              
                                                                                                  
 block3b_activation (Activation  (None, None, None,   0          ['block3b_bn[0][0]']             
 )                              240)                                                              
                                                                                                  
 block3b_se_squeeze (GlobalAver  (None, 240)         0           ['block3b_activation[0][0]']     
 agePooling2D)                                                                                    
          

 block4b_activation (Activation  (None, None, None,   0          ['block4b_bn[0][0]']             
 )                              480)                                                              
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block4b_se_squeeze[0][0]']     
                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_s

                                                                                                  
 block5a_activation (Activation  (None, None, None,   0          ['block5a_bn[0][0]']             
 )                              480)                                                              
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
          

 agePooling2D)                                                                                    
                                                                                                  
 block5c_se_reshape (Reshape)   (None, 1, 1, 672)    0           ['block5c_se_squeeze[0][0]']     
                                                                                                  
 block5c_se_reduce (Conv2D)     (None, 1, 1, 28)     18844       ['block5c_se_reshape[0][0]']     
                                                                                                  
 block5c_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5c_se_reduce[0][0]']      
                                                                                                  
 block5c_se_excite (Multiply)   (None, None, None,   0           ['block5c_activation[0][0]',     
                                672)                              'block5c_se_expand[0][0]']      
          

                                                                                                  
 block6b_se_expand (Conv2D)     (None, 1, 1, 1152)   56448       ['block6b_se_reduce[0][0]']      
                                                                                                  
 block6b_se_excite (Multiply)   (None, None, None,   0           ['block6b_activation[0][0]',     
                                1152)                             'block6b_se_expand[0][0]']      
                                                                                                  
 block6b_project_conv (Conv2D)  (None, None, None,   221184      ['block6b_se_excite[0][0]']      
                                192)                                                              
                                                                                                  
 block6b_project_bn (BatchNorma  (None, None, None,   768        ['block6b_project_conv[0][0]']   
 lization)

 block6d_se_reduce (Conv2D)     (None, 1, 1, 48)     55344       ['block6d_se_reshape[0][0]']     
                                                                                                  
 block6d_se_expand (Conv2D)     (None, 1, 1, 1152)   56448       ['block6d_se_reduce[0][0]']      
                                                                                                  
 block6d_se_excite (Multiply)   (None, None, None,   0           ['block6d_activation[0][0]',     
                                1152)                             'block6d_se_expand[0][0]']      
                                                                                                  
 block6d_project_conv (Conv2D)  (None, None, None,   221184      ['block6d_se_excite[0][0]']      
                                192)                                                              
                                                                                                  
 block6d_p

                                                                                                  
 decoder_stage0b_bn (BatchNorma  (None, None, None,   1024       ['decoder_stage0b_conv[0][0]']   
 lization)                      256)                                                              
                                                                                                  
 decoder_stage0b_relu (Activati  (None, None, None,   0          ['decoder_stage0b_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 decoder_stage1_upsampling (UpS  (None, None, None,   0          ['decoder_stage0b_relu[0][0]']   
 ampling2D)                     256)                                                              
                                                                                                  
 decoder_s

 on)                            32)                                                               
                                                                                                  
 decoder_stage4_upsampling (UpS  (None, None, None,   0          ['decoder_stage3b_relu[0][0]']   
 ampling2D)                     32)                                                               
                                                                                                  
 decoder_stage4a_conv (Conv2D)  (None, None, None,   4608        ['decoder_stage4_upsampling[0][0]
                                16)                              ']                               
                                                                                                  
 decoder_stage4a_bn (BatchNorma  (None, None, None,   64         ['decoder_stage4a_conv[0][0]']   
 lization)                      16)                                                               
          

In [9]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score



H = 224
W = 224

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                ## (1, 256, 256, 3)


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                    ## (256, 256)
    return ori_x, x

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)  ## (256, 256, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) ## (256, 256, 3)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving results """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")

    """ Load the test data """
    dataset_path = "C:/Users/Haris/ICAI_Paper/Dataset17"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Exctracting the image name """
        name = x.split("/")[-1]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Predicting the mask """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        """ Saving the predicted mask """
        save_image_path = f"results/{name}"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ mean metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 215/215 [00:54<00:00,  3.96it/s]

Accuracy: 0.96607
F1: 0.89391
Jaccard: 0.82418
Recall: 0.90098
Precision: 0.91627
